<p style="color: #000000; font-size: 32px; font-weight: bold; text-align: center; margin-top: 20px;">  En la Búsqueda de los Próximos 200 Pozos Petroleros 
</p>
<p style="color: #000000; font-size: 24px; text-align: center; margin-bottom: 20px;"> Oily Giant Company
</p>

<hr style="border: .4px solid #000000; width: 55%; margin: 10px auto;">

**Este proyecto** se centra en identificar las mejores ubicaciones para abrir nuevos pozos de petróleo en tres regiones utilizando datos geológicos sintéticos y un modelo de regresión lineal. El objetivo principal es maximizar los beneficios y minimizar los riesgos, asegurando la sostenibilidad económica de la inversión.

**Los principales pasos a realizar son los siguientes:**  

1. **Leer** los archivos con los parámetros de los pozos petrolíferos: `calidad del crudo` y `volumen de reservas`.  
2. **Crear** un modelo para predecir el `volumen de reservas` en nuevos pozos.  
3. **Seleccionar** los pozos con los valores estimados más altos.  
4. **Identificar** la región con el mayor beneficio total para los pozos seleccionados.  

Se analizan los beneficios y riesgos potenciales utilizando la técnica **bootstrapping**.


**Las siguientes son las condiciones para realizar el proyectop:**
  - Solo se debe usar la regresión lineal para el entrenamiento del modelo.
  - Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
  - El presupuesto para el desarrollo de 200 pozos petroleros es de **100 millones de dólares**.
  - Un barril de materias primas genera **4.5 USD** de ingresos. El ingreso de una unidad de producto es de **4500 dólares** (el volumen de reservas está expresado en miles de barriles).
  - Después de la evaluación de riesgo:
    - Mantén solo las regiones con un riesgo de pérdidas inferior al **2.5%**.
    - De las que se ajustan a los criterios, selecciona la región con el beneficio promedio más alto.

## Descripción de los Datos
Los datasets de las tres regiones contienen las siguientes características:

- **`id`**: Identificador único de pozo.
- **`f0`, `f1`, `f2`**: Características de los puntos de exploración.
- **`product`**: Volumen de reservas de petróleo (miles de barriles).
---  
**Los datos son sintéticos:** los detalles del contrato y las características del pozo no se publican

# 1. Librerias y Carga de Datos

In [24]:
import pandas as pd
import numpy as np
from numpy import mean
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import locale # formatting currency

In [25]:
# Leer los archivos CSV
geo_0 = pd.read_csv("geo_data_0.csv")
geo_1 = pd.read_csv("geo_data_1.csv")
geo_2 = pd.read_csv("geo_data_2.csv")

In [36]:
# Crear la lista de datasets
datasets = [geo_0, geo_1, geo_2]

# Imprimir las primeras 5 filas de cada dataset
for i in range(len(datasets)):
    print('geo_' + str(i) + ':')
    print(datasets[i].head(5))
    print()

geo_0:
         f0        f1        f2     product
0  0.705745 -0.497823  1.221170  105.280062
1  1.334711 -0.340164  4.365080   73.037750
2  1.022732  0.151990  1.419926   85.265647
3 -0.032172  0.139033  2.978566  168.620776
4  1.988431  0.155413  4.751769  154.036647

geo_1:
          f0         f1        f2     product
0 -15.001348  -8.276000 -0.005876    3.179103
1  14.272088  -3.475083  0.999183   26.953261
2   6.263187  -5.948386  5.001160  134.766305
3 -13.081196 -11.506057  4.999415  137.945408
4  12.702195  -8.147433  5.004363  134.766305

geo_2:
         f0        f1        f2     product
0 -1.146987  0.963328 -0.828965   27.758673
1  0.262778  0.269839 -2.530187   56.069697
2  0.194587  0.289035 -5.586433   62.871910
3  2.236060 -0.553760  0.930038  114.572842
4 -0.515993  1.716266  5.899011  149.600746



In [37]:
# Info. de datasets
for i in range(len(datasets)):
    print('geo_' + str(i) + ' info:')
    print(datasets[i].info())
    print()

geo_0 info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None

geo_1 info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None

geo_2 info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  

**Calidad de los datos:**
  - No hay valores ausentes.
  - Los tipos de datos (*dtypes*) son correctos.
  - Se buscarán valores duplicados.

In [28]:
# Duplicados en los datasets
print(f"Duplicados en Geo 0: {geo_0.duplicated().sum()}")
print(f"Duplicados en Geo 1: {geo_1.duplicated().sum()}")
print(f"Duplicados en Geo 2: {geo_2.duplicated().sum()}")

Duplicados en Geo 0: 0
Duplicados en Geo 1: 0
Duplicados en Geo 2: 0


Debido a que no son datos necesarios, se eliminaran las columnas de `'id'` en los 3 DataFrames. 

In [29]:
# Eliminar columnas de 'id' 
geo_0 = geo_0.drop('id',axis=1)
geo_1 = geo_1.drop('id',axis=1)
geo_2 = geo_2.drop('id',axis=1)

# 2. Entrenamiento y Prueba del Modelo para cada Región.

Ahora, se crean los juntos entrenamiento y validación para las tres regiones.    
La **Relación es de 3:1**, entonces es un 75% para entrenamiento y un 25% para validacion.   
Para ello, se usará la función `train_test_split` del módulo `sklearn.model_selection` para dividir los datos.   

In [30]:
# Crear training, validation, y testing datasets (Ratio of 3:1)

# 'Features' para cada región
geo_0_features = geo_0.drop('product', axis=1)
geo_1_features = geo_1.drop('product', axis=1)
geo_2_features = geo_2.drop('product', axis=1)

# 'Targets' para cada región
geo_0_target = geo_0['product']
geo_1_target = geo_1['product']
geo_2_target = geo_2['product']

# Dividir feature and target para cada region en training y validation datasets 
# test_size con 0.25 y training dataset con 0.75 de los datos
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(geo_0_features, geo_0_target, test_size=0.25)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(geo_1_features, geo_1_target, test_size=0.25)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(geo_2_features, geo_2_target, test_size=0.25)

In [31]:
# Función para verificar proporciones de las divisiones de datos para cada región
def check_split_proportion(region_name, total, train, valid, test_size=0.25):
    train_prop = train / total
    valid_prop = valid / total
    print(f"{region_name}:")
    print(f"Entrenamiento: {train_prop:.2f} | Validación: {valid_prop:.2f}")

# Validar proporciones para cada región
check_split_proportion("Geo 0", len(geo_0_features), len(features_train_0), len(features_valid_0))
check_split_proportion("Geo 1", len(geo_1_features), len(features_train_1), len(features_valid_1))
check_split_proportion("Geo 2", len(geo_2_features), len(features_train_2), len(features_valid_2))

Geo 0:
Entrenamiento: 0.75 | Validación: 0.25
Geo 1:
Entrenamiento: 0.75 | Validación: 0.25
Geo 2:
Entrenamiento: 0.75 | Validación: 0.25


## 2.1. Entrenamiento de Modelo  
Se creará una función para entrenar el modelo y analizar los resultados.

- `sklearn.linear_model.LinearRegression`: Se utiliza para implementar el modelo de regresión lineal, que predice las reservas en función de las características de los datos.   

- `sklearn.metrics.mean_squared_error`: Calcula el Error Cuadrático Medio (MSE), una métrica que evalúa el rendimiento del modelo y su precisión en las predicciones.    

Entrenar un modelo separado para cada región permite capturar variaciones específicas en los datos, lo que puede mejorar la precisión de las predicciones.

In [32]:
# Función para entrenar el modelo, realizar predicciones y calcular métricas
def analizar_reservas(features_train, target_train, features_valid, target_valid):
    # Crear el modelo
    modelo = LinearRegression()
    
    # Entrenar el modelo con los datos de entrenamiento
    modelo.fit(features_train, target_train)
    
    # Realizar predicciones en los datos de validación
    predicciones = modelo.predict(features_valid)
    
    # Calcular el promedio de reservas previstas
    promedio_reservas = round(predicciones.mean(), 2)
    
    # Calcular el RMSE del modelo
    rmse = mean_squared_error(target_valid, predicciones) ** 0.5
    
    # Imprimir resultados
    print(f"Volumen promedio de reservas previstas: {promedio_reservas} mil barriles")
    print(f"RMSE del modelo de regresión lineal: {round(rmse, 2)}")
    
    # Devolver el modelo entrenado y las predicciones
    return modelo, predicciones

# Aplicar la función a las tres regiones
print("Geo 0")
modelo_0, predicciones_0 = analizar_reservas(features_train_0, target_train_0, features_valid_0, target_valid_0)

print("\nGeo 1")
modelo_1, predicciones_1 = analizar_reservas(features_train_1, target_train_1, features_valid_1, target_valid_1)

print("\nGeo 2")
modelo_2, predicciones_2 = analizar_reservas(features_train_2, target_train_2, features_valid_2, target_valid_2)


Geo 0
Volumen promedio de reservas previstas: 92.6 mil barriles
RMSE del modelo de regresión lineal: 37.65

Geo 1
Volumen promedio de reservas previstas: 68.8 mil barriles
RMSE del modelo de regresión lineal: 0.89

Geo 2
Volumen promedio de reservas previstas: 95.21 mil barriles
RMSE del modelo de regresión lineal: 39.98


**Análisis de Resultados**

- **Geo 1**: Muestra el mejor rendimiento del modelo con el error más bajo (**RMSE = 0.89**) y predice reservas promedio de **68.48 mil barriles**.    
- **Geo 0 y Geo 2**: Tienen errores de predicción mucho más altos (**RMSE de 37.75 y 39.92**, respectivamente) y predicen reservas más altas (**92.78 y 95.05 mil barriles**).     

El **RMSE significativamente más bajo para Geo 1** sugiere que las predicciones para esta área son mucho más confiables que las de las otras dos regiones.   


# 3. Cálculo de Ganancias  

Se tomarán en cuenta los siguientes datos clave para calcular las ganancias:   

- **Presupuesto**: El presupuesto total para desarrollar 200 pozos petroleros es de **100 millones de dólares**, lo que equivale a **0.5 millones por pozo**.   
- **Ingresos por barril**: Cada barril de materias primas genera **4.5 USD** de ingresos.   
- **Ingresos por unidad de producto**: Como el volumen de reservas se expresa en miles de barriles, el ingreso por unidad de producto es de **4500 dólares** (1000 barriles × 4.5 USD/barril).   

In [33]:
# Variables para presupuesto
budget = 100_000_000  # en dólares
profit = 4_500      # ingresos por unidad (miles de barriles)

# Volumen de reservas necesario para no tener pérdidas
reservas_necesarias = budget / profit

# Resultados
print(f"Se requieren un total de {round(reservas_necesarias, 2)} miles de barriles para evitar pérdidas.")
print(f"Se requieren aproximadamente {round(reservas_necesarias / 200, 2)} miles de barriles por reserva.")

Se requieren un total de 22222.22 miles de barriles para evitar pérdidas.
Se requieren aproximadamente 111.11 miles de barriles por reserva.


**Analizando la relación entre los resultados**:

Las predicciones por área geográfica muestran que ninguna zona por sí sola alcanza el requerimiento de 111.11 mil barriles promedio por pozo que se necesita.

Ninguna zona individualmente alcanza el volumen promedio mínimo requerido por pozo.

## 3.1 Análisis de Ganancias, Predicciones y Evaluación de Riesgos en Pozos Petrolero

Para finalizar con el análisis, se calculan las posibles ganancias seleccionando los pozos más productivos según las predicciones del modelo de regresión lineal. Para cada región, se estima el volumen de reservas, se proyectan los beneficios y se sugiere la región más rentable para desarrollar pozos, basado en los resultados obtenidos.

Además, se usa la técnica de **bootstrapping** con 1,000 simulaciones para evaluar los riesgos y beneficios. Esto incluye calcular el beneficio promedio, el intervalo de confianza del 95% y la probabilidad de pérdidas. 

In [34]:
# Función para calcular el beneficio basado en los valores predichos de cada región
def get_profit(targets, predictions, count, budget):
    '''Función para calcular el beneficio potencial basado en los volúmenes de reservas de petróleo predichos para cada región'''
    
    # Ordenar los volúmenes predichos de mayor a menor
    predictions_sorted = pd.Series(predictions).sort_values(ascending=False)[:count]
    
    # Seleccionar los 200 volúmenes de reservas más grandes para cada región, pero utilizar los volúmenes objetivo (volúmenes reales)
    selected_wells = targets.iloc[predictions_sorted.index]
    
    # Calcular el beneficio basado en una unidad que produce un ingreso de $4,500
    # Restar el presupuesto de $100,000,000 del ingreso total
    profit = (4500 * selected_wells.sum()) - budget
    
    # Retornar el valor del beneficio
    return round(profit, 2)


# Crear una lista de los valores predichos y los valores reales (targets) para cada región
predictions = [predicciones_0, predicciones_1, predicciones_2]
targets = [target_valid_0.reset_index(drop=True), target_valid_1.reset_index(drop=True), target_valid_2.reset_index(drop=True)]

profits = []

# Bucle para ejecutar la función get_profit en los conjuntos de datos predichos de cada región
# Almacenar los valores de beneficio en la lista 'profits'
for i in range(len(predictions)):
    profits.append(get_profit(targets[i], predictions[i], 200, budget))  # Asegúrate de pasar 'budget' como argumento

# Establecer la moneda local en USD (asegúrate de utilizar un locale que soporte formato monetario)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Imprimir los valores de beneficio para cada región
for i in range(len(profits)):
    print(f'Geo {i}')
    print(f'Beneficio: {locale.currency(profits[i], grouping=True)}')
    print(f'Volumen de reservas objetivo: {round((profits[i] + budget) / 4500, 2)} miles de barriles\n')

Geo 0
Beneficio: $31,526,120.57
Volumen de reservas objetivo: 29228.03 miles de barriles

Geo 1
Beneficio: $24,150,866.97
Volumen de reservas objetivo: 27589.08 miles de barriles

Geo 2
Beneficio: $26,092,896.14
Volumen de reservas objetivo: 28020.64 miles de barriles



In [35]:
def get_profit_distribution(targets, predictions, count, budget):
    '''Función que calcula la distribución del beneficio utilizando el método de bootstrapping'''
    
    # Inicializar el estado aleatorio
    state = np.random.RandomState(12345)
    
    # Lista para almacenar los valores de los beneficios
    values = []
    
    # Crear un DataFrame con las predicciones y los objetivos
    combined_df = pd.DataFrame()
    combined_df['predictions'] = predictions
    combined_df['targets'] = targets.reset_index(drop=True)
    
    # Obtener 1000 muestras con reemplazo
    for i in range(1000):
        target_subsample = combined_df.sample(n=500, replace=True, random_state=state).reset_index(drop=True)
        values.append(get_profit(target_subsample['targets'], target_subsample['predictions'], count, budget))
    
    # Convertir la lista de beneficios en una serie de pandas
    values = pd.Series(values)
    
    # Calcular la media de los beneficios
    mean = values.mean()
    
    # Intervalo de confianza del 95%
    upper = values.quantile(0.975)
    lower = values.quantile(0.025)
    
    # Riesgo de pérdida (beneficio negativo)
    count_loss = (values < 0).sum()
    risk_of_loss = (count_loss * 100) / len(values)
    
    # Calcular el volumen objetivo de reservas de petróleo
    target_oil_reserve_volume = (mean + budget) / 4500
    
    # Imprimir los resultados
    print(f"Ganacias promedio: {locale.currency(mean, grouping=True)}")
    print(f"Volumen objetivo de reservas de petróleo: {round(target_oil_reserve_volume, 2)} miles de barriles")
    print(f"Intervalo de confianza al 95%: {locale.currency(lower, grouping=True)} to {locale.currency(upper, grouping=True)}")
    print(f"Riesgo de pérdida: {risk_of_loss:.2f}%")

# Ejecutar la función para cada región
for i in range(len(predictions)):
    print(f"Geo {i}")
    get_profit_distribution(targets[i], predictions[i], 200, budget)
    print()

Geo 0
Ganacias promedio: $4,063,083.37
Volumen objetivo de reservas de petróleo: 23125.13 miles de barriles
Intervalo de confianza al 95%: -$843,254.51 to $9,110,175.60
Riesgo de pérdida: 5.20%

Geo 1
Ganacias promedio: $4,547,122.26
Volumen objetivo de reservas de petróleo: 23232.69 miles de barriles
Intervalo de confianza al 95%: $659,233.98 to $8,829,539.72
Riesgo de pérdida: 1.20%

Geo 2
Ganacias promedio: $3,238,367.33
Volumen objetivo de reservas de petróleo: 22941.86 miles de barriles
Intervalo de confianza al 95%: -$1,980,487.95 to $8,315,987.85
Riesgo de pérdida: 11.80%



# Conclusiones

Analizando los datos proporcionados, es más recomendable realizar los 200 pozos en la **Región 1 (geo_1)** por las siguientes razones:

1. **Beneficio Promedio más Alto**: El beneficio promedio para la Geo 1 es de **4,547,122.26 USD**, el más alto entre las tres regiones (Geo 0: 4,063,083.37 USD y Geo 2: 3,238,367.33 USD). Esto sugiere que, en promedio, la Geo 1 generaría un mayor retorno de inversión.

2. **Volumen de Reservas Objetivo Sostenible**: La Geo 1 tiene un volumen de reservas objetivo de **23,232.69 miles de barriles**, lo que es razonablemente cercano a las otras dos regiones, pero con un beneficio promedio más alto. Esto implica que, aunque las reservas de petróleo en la Geo 1 son relativamente similares en tamaño a las de las otras dos, la capacidad de generar más ganancias con un volumen de reservas similar es un punto a favor.

3. **Riesgo de Pérdida más Bajo**: El riesgo de pérdida en la Geo 1 es solo **1.20%**, el más bajo entre las tres regiones. Comparado con la Geo 0 (5.20%) y la Geo 2 (11.80%), la Geo 1 presenta una menor probabilidad de pérdidas significativas, lo que la convierte en una opción más segura para la inversión.

4. **Intervalo de Confianza al 95%**: En la Geo 1, el intervalo de confianza al 95% va de **659,233.98 USD a 8,829,539.72 USD**, lo que indica una probabilidad de resultados positivos relativamente alta y un rango de resultados que no cae por debajo de 659,233.98 USD. En contraste, las otras dos regiones tienen intervalos de confianza con valores negativos en su rango inferior, lo que sugiere que hay una mayor incertidumbre y potencial para pérdidas en esas áreas.

En resumen, la Geo 1 tiene una combinación de un beneficio promedio más alto, un volumen de reservas objetivo competitivo, un bajo riesgo de pérdida y una distribución de resultados más favorable, lo que la convierte en la opción más atractiva para realizar los 200 pozos.